In [10]:
import pandas as pd
import re
import string
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense,Activation,Conv1D, Dropout,MaxPooling1D,GlobalMaxPooling1D,BatchNormalization,Embedding,LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split

In [3]:
train=pd.read_csv('D:/toxic-comment/train.csv')
test=pd.read_csv('D:/toxic-comment/test.csv')

In [4]:
train['comment_text']=train['comment_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
tokenizer = Tokenizer(num_words=None,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True,
                      split=" ",
                      char_level=False)
tokenizer.fit_on_texts(train['comment_text'])
tokenized_train = tokenizer.texts_to_sequences(train['comment_text'])
tokenized_test = tokenizer.texts_to_sequences(test['comment_text'])
word_index = tokenizer.word_index
len(word_index)

206001

In [5]:
tokenized_train= pad_sequences(tokenized_train, maxlen=150, padding='post')
tokenized_test= pad_sequences(tokenized_test, maxlen=150, padding='post')

In [6]:
embedding_dim=300
embeddings_index={}
f = open('D:/wiki.en.vec', encoding="utf8")
for line in f:
    values=line.rstrip().rsplit(' ',embedding_dim)
    word=values[0]
    c=np.asarray(values[1:],dtype='float32')
    embeddings_index[word] = c
f.close()
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [7]:
vocab_size=len(word_index)
vocab_size

206001

In [ ]:
X =tokenized_train
X_test = tokenized_test
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
X_train, X_eval, y_train ,y_eval = train_test_split(X, y,test_size=0.05,shuffle=True)

In [9]:
max_len=150
model=Sequential()
model.add(Embedding(vocab_size + 1, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=True))
model.add(Conv1D(filters=128,kernel_size=5,padding='same',activation='relu'))
model.add(MaxPooling1D(3))
model.add(GlobalMaxPooling1D())
model.add(BatchNormalization())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(6, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='sgd')
model.summary()
tokenized_train=np.array(tokenized_train)
model.fit(X_train,y_train,validation_data=(X_eval, y_eval),
                    epochs=40,verbose=1,batch_size=128)
model.predict(X_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 300)          61800600  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 150, 128)          192128    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 50, 128)           0         
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dense_1 (Dense)              (None, 50)                6450      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
__________

KeyboardInterrupt: 

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size + 1, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=True))
model.add(LSTM(60, return_sequences=True, name='lstm_layer'))
model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(3))
model.add(GlobalMaxPooling1D())
model.add(BatchNormalization())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(6, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='sgd')
model.summary()
tokenized_train=np.array(tokenized_train)
model.fit(X_train,y_train,validation_data=(X_eval, y_eval),
                    epochs=2,verbose=1,batch_size=128)
model.predict(X_test)